In [ ]:
using LinearAlgebra, Random, GLMakie


dt = 0.1
n_steps = 500
process_noise_std = 0.05
measurement_noise_std = 0.4

# State transition and observation matrices
F = [1.0 dt; 0.0 1.0]
H = [1.0 0.0]

Q = [0.25 0.5; 0.5 1.0] .* process_noise_std^2
R = [measurement_noise_std^2]

# sine wave - sample true signal
t = 0:dt:(n_steps*dt)
true_pos = @. sin(t)
true_vel = @. cos(t)
true_states = [[true_pos[i], true_vel[i]] for i in eachindex(t)]

#  Generate noisy measurements 
Random.seed!(42)
measurements = [x[1] + randn() * measurement_noise_std for x in true_states]

#  Initialize Kalman Filter 
x_est = [0.0, 1.0]  # initial estimate (position, velocity)
P = Matrix(I, 2, 2) .* 1.0
estimates = [x_est]

# For plotting additional metrics
kalman_gains = []
covariances = []
errors = []

# Main Kalman loop 
for k in 2:length(t)
    # Prediction
    x_pred = (F * estimates[end])[:]   
    P_pred = F * P * F' + Q
    
    # Update
    z = measurements[k]
    K = P_pred * H' * inv(H * P_pred * H' + R)
    innovation = z - (H * x_pred)[1]
    x_est = (x_pred + (K * innovation))[:]  # flatten
    P = (I - K * H) * P_pred
    
    push!(estimates, x_est)
    
    # Save metrics
    push!(kalman_gains, K[1])                   # position gain
    push!(covariances, P[1,1])                  # position variance
    push!(errors, abs(true_pos[k] - x_est[1])) # tracking error
end

# get estimates for plotting 
est_pos = [x[1] for x in estimates]
est_vel = [x[2] for x in estimates]

#  Visualization 

# True signal vs measurement vs Kalman estimate
fig1 = Figure(resolution = (800, 500))
ax1 = Axis(fig1[1, 1], xlabel="Time (s)", ylabel="Position", 
           title="Kalman Filter Tracking a Noisy Sine Wave")
GLMakie.lines!(ax1, t, true_pos, color=:green, linewidth=3, label="True Signal")
GLMakie.scatter!(ax1, t, measurements, color=:red, markersize=3, label="Noisy Measurements")
GLMakie.lines!(ax1, t, est_pos, color=:blue, linewidth=2, label="Kalman Estimate")
axislegend(ax1; position=:rt)
save("kalman_tracking.png", fig1)

fig1

# Kalman Gain Evolution
fig2 = Figure(resolution = (800, 400))
ax2 = Axis(fig2[1, 1], xlabel="Time (s)", ylabel="Kalman Gain (Position)", 
           title="Kalman Gain Evolution")
GLMakie.lines!(ax2, t[2:end], kalman_gains, color=:purple, linewidth=2)
save("kalman_gain.png", fig2)

fig2


#  Position Uncertainty (variance)
fig3 = Figure(resolution = (800, 400))
ax3 = Axis(fig3[1, 1], xlabel="Time (s)", ylabel="Variance (P[1,1])", 
           title="Position Uncertainty Over Time")
GLMakie.lines!(ax3, t[2:end], covariances, color=:orange, linewidth=2)
save("kalman_variance.png", fig3)

fig3

#  Tracking Error
fig4 = Figure(resolution = (800, 400))
ax4 = Axis(fig4[1, 1], xlabel="Time (s)", ylabel="Absolute Error", 
           title="Error Between True Position and Estimate")
GLMakie.lines!(ax4, t[2:end], errors, color=:red, linewidth=2)
save("kalman_error.png", fig4)

fig4
